In [1]:
import pandas as pd
import os
import csv
import numpy as np
from scipy import stats
from matplotlib import pyplot as plt
from scipy.ndimage.filters import median_filter
%matplotlib

Using matplotlib backend: Qt5Agg


In [2]:
#en Experimentos está FLIR y en Experimentos2 están temp y tempcup y temp4cup
from Experimentos2 import Experimentos, labelLines

[Errno 2] No such file or directory: '.\\Data\\5cc50C - flir.csv'
[Errno 2] No such file or directory: '.\\Data\\5cc50C - flir.csv'
[Errno 2] No such file or directory: '.\\Data\\5cc50C - max av.csv'
[Errno 2] No such file or directory: '.\\Data\\5cc50C - fspot.csv'
[Errno 2] No such file or directory: '.\\Data\\5cc50C - max av cup.csv'
[Errno 2] No such file or directory: '.\\Data\\1cc54C - flir.csv'
[Errno 2] No such file or directory: '.\\Data\\1cc54C - flir.csv'
[Errno 2] No such file or directory: '.\\Data\\1cc54C - max av.csv'
[Errno 2] No such file or directory: '.\\Data\\1cc54C - fspot.csv'
[Errno 2] No such file or directory: '.\\Data\\1cc54C - max av cup.csv'
[Errno 2] No such file or directory: '.\\Data\\25cc54C - flir.csv'
[Errno 2] No such file or directory: '.\\Data\\25cc54C - flir.csv'
[Errno 2] No such file or directory: '.\\Data\\25cc54C - max av.csv'
[Errno 2] No such file or directory: '.\\Data\\25cc54C - fspot.csv'
[Errno 2] No such file or directory: '.\\Data\\25cc

In [129]:
#experimentos sin FLIR: 11
#experimentos con FLIR: 22
for i, exp in enumerate(Experimentos):
   print(i, exp, (exp.flir is not None))

0 5cc50C.csv False
1 1cc54C.csv False
2 25cc54C.csv False
3 10cc60C.csv False
4 25cc60C.csv False
5 1cc59C.csv False
6 1cc47C.csv True
7 1cc55C.csv True
8 25cc58C.csv True
9 25cc70C.csv True
10 10cc67C.csv True
11 10cc59C.csv True
12 5cc63C.csv False
13 5cc59C.csv True
14 10cc49C.csv True
15 25cc62C.csv False
16 1cc62C.csv False
17 5cc53C.csv True
18 5a25cc56C.csv True
19 5cc57C.csv True
20 25cc50C.csv True
21 10cc55C.csv True
22 1cc24C.csv False
23 10cc57C.csv True
24 10cc24C.csv False
25 5cc55C.csv True
26 5cc58C.csv True
27 1cc48C.csv True
28 1cc51C.csv True
29 5cc48C.csv True
30 10cc50C.csv True
31 1cc54,7C.csv True
32 1cc52C.csv True


In [3]:
numeropixeles = 1020100
rho_manjar = 1340
kilos_manjar = 2

In [4]:
#genero ESPESORES a partir de AREAS aproximando el flujo a un ELIPSOIDE REAL. 
for exp in Experimentos:
    if exp.flir is not None:
        try:
            exp.flir['area_m2'] = exp.flir['FLIR: area']/numeropixeles
            exp.flir['volumen_emitido'] = exp.param['caudal'][0]*exp.flir['FLIR: tiempo']
        except:
            continue
        volumen_total = kilos_manjar/rho_manjar
        exp.flir['volumen_emitido'][exp.flir['volumen_emitido'] > volumen_total] = volumen_total
        exp.flir['espesor area'] = 3*exp.flir['volumen_emitido']/(2*exp.flir['area_m2'])
        exp.dinamica['area a*b'] = 3.14*(exp.dinamica['avance: largo total flujo']/2)*(exp.dinamica['avance: ancho maximo']/2)
        exp.dinamica['volumen a*b'] = exp.param['caudal'][0]*exp.dinamica['avance: tiempo']
        exp.dinamica['volumen a*b'][exp.dinamica['volumen a*b'] > volumen_total] = volumen_total
        exp.dinamica['espesor largoancho'] = 3*exp.dinamica['volumen a*b']/(2*exp.dinamica['area a*b'])


C:\Users\CCVV\Anaconda3b\lib\site-packages\ipykernel\__main__.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
C:\Users\CCVV\Anaconda3b\lib\site-packages\ipykernel\__main__.py:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


In [41]:
#Añado velos a params x facilidad
for exp in Experimentos:
    for i in range(33):
        if exp.param['NOMBRE'][0]==velos.iloc[i,:]['nombre']:
            exp.param['v1'] = velos.iloc[i,:]['v1']
            exp.param['v2'] = velos.iloc[i,:]['v2']
            exp.param['L*'] = velos.iloc[i,:]['L*']
            exp.param['t*'] = velos.iloc[i,:]['t*']

In [5]:
#achico los rangos
for exp in Experimentos:
    if exp.flir is not None:
        try:
            exp.flir['rango 1'] = exp.flir['FLIR: rango 1']+exp.flir['FLIR: rango 2']+exp.flir['FLIR: rango 3']
            exp.flir['rango 2'] = exp.flir['FLIR: rango 4']+exp.flir['FLIR: rango 5']+exp.flir['FLIR: rango 6']
            exp.flir['rango 3'] = exp.flir['FLIR: rango 7']+exp.flir['FLIR: rango 8']+exp.flir['FLIR: rango 9']

        except:
            continue

In [90]:
listarangos = []

for exp in Experimentos:
    if exp.flir is not None:
        try:
            holder = {} #es un diccionario 
            holder['nombre'] = exp.param['NOMBRE'][0]#solo el valor de la primera fila
            holder['rango 1'] = exp.flir['rango 1']#toda la columna
            holder['rango 2'] = exp.flir['rango 2']#toda la columna
            holder['rango 3'] = exp.flir['rango 3']#toda la columna
            holder['avance'] = exp.dinamica['avance: largo total flujo']#toda la columna
            holder['tiempoavance'] = exp.dinamica['avance: tiempo']#toda la columna
            holder['tiempoflir'] = exp.flir['FLIR: tiempo']#toda la columna
            holder['caudal'] = exp.param['caudal'][0]#solo el valor de la primera fila
            holder['angulo'] = exp.param['angulo inclinacion'][0]
            holder['uso'] = exp.param['uso manjar'][0]

            listarangos.append(holder)
        except Exception as e:
            print(e)

In [125]:
#grafico de rangos por caudal

styles= ['red', 'sienna', 'darkgreen', 'purple', 'darkcyan', 'coral', 'navy', 
        'springgreen', 'magenta', 'y', 'palevioletred', 'blue']

fig = plt.figure()
axes = []


for x in range(c):
    axes.append(fig.add_subplot(2, 3, x + 1))

for i, (key, group) in enumerate(groupby(sorted(listarangos, key=lambda x: x['caudal']), key=lambda x: x['caudal'])):
    print(key, c)
    if key == 1e-06:
        c = 0     
        for exp in group:
            colors = styles[c]
            ax.set_title(exp['nombre'])

            axes[c].plot(exp['tiempo'], exp['rango 1'], color=colors)
            axes[c].plot(exp['tiempo'], exp['rango 2'], color=colors)
            axes[c].plot(exp['tiempo'], exp['rango 3'], color=colors)
            #axes[c].plot(exp['otrotiempo'], exp['distancia'], color=colors)
           
            c = c + 1
        break
    
fig.show()

1e-06 6


In [25]:
for exp in Experimentos:
    if exp.flir is not None:
        try:
            print(exp.flir['rango 1'].max())
            print(exp.flir['rango 2'].max())
            print(exp.flir['rango 3'].max())

        except:
            continue

1.0
0.747871812
0.619908466
0.565432179
0.846303847
0.699669967
1.000000001
0.93413261
0.90354621
1.000000001
0.934428167
0.906165947
0.999959626
0.934389829
0.898852245
0.999919647
0.970162376
0.859032918
1.000000001
0.951593036
0.832425786
0.999970252
0.928591873
0.848566546
0.60106383
0.939085894
0.85113972
1.000000001
0.94198386
0.752496543
0.662088922
0.901257787
0.812531838
0.689476333
0.944947648
0.855160069
1.000000001
0.93358557
0.903052395
1.000000001
0.956743889
0.887895111
1.000000038
0.795114777
0.465571764
1.000000001
0.933673238
0.796978281
0.820458661
0.770635781
0.72947053
1.000000044
0.933196364
0.771124557
1.000000001
0.907328555
0.807389334
1.000000001
0.943459649
0.835539401
0.725556153
0.870059399
0.746544606
0.901354672
0.832376863
0.699649738


In [23]:
#GENERA PATCHES DE COLORES SEGUN TEMPERATURA
import matplotlib.patches as mpatches

pat1 = mpatches.Patch(color='C1', label='temp < 40')
pat2 = mpatches.Patch(color='C2', label='temp < 45')
pat3 = mpatches.Patch(color='C3', label='temp < 50')
pat4 = mpatches.Patch(color='C4', label='temp < 55')
pat5 = mpatches.Patch(color='C5', label='temp < 60')
pat6 = mpatches.Patch(color='C6', label='temp < 65')
pat7 = mpatches.Patch(color='C7', label='temp < 70')

numerocolores = 7
colors = ['pat{}'.format(i+1) for i in range(numerocolores)]

print(colors)

['pat1', 'pat2', 'pat3', 'pat4', 'pat5', 'pat6', 'pat7']


In [5]:
velos = pd.read_excel('Velocidades y punto.xlsx')

In [6]:
for exp in Experimentos:
    exp.param['tiempo fin eyeccion'] = exp.param['volumen total teorico']/exp.param['caudal']

In [7]:
#LISTA para ESPESOR vs TIEMPO
listaporcaudal = []

for exp in Experimentos:
    if exp.flir is not None:
        try:
            holder = {} #es un diccionario 
            holder['nombre'] = exp.param['NOMBRE'][0]#solo el valor de la primera fila
            holder['espesor a'] = exp.flir['espesor area']#toda la columna
            holder['espesor b'] = exp.dinamica['espesor largoancho']
            holder['tiempo a'] = exp.flir['FLIR: tiempo']#toda la columna
            holder['tiempo b'] = exp.dinamica['avance: tiempo']
            holder['caudal'] = exp.param['caudal'][0]#solo el valor de la primera fila
            holder['angulo'] = exp.param['angulo inclinacion'][0]
            holder['uso'] = exp.param['uso manjar'][0]
            holder['teyec'] = exp.param['tiempo fin eyeccion'][0]
            holder['espesorfinal'] = exp.perfil['perfil: espesor'].max()
            holder['temperatura'] = exp.param['t max'][0]

            listaporcaudal.append(holder)
        except Exception as e:
            print(e)

In [98]:
for exp in listaporcaudal:
    print(len(exp['espesor b']), len(exp['tiempo b']))

30 30
32 32
40 40
39 39
33 33
31 31
39 39
40 40
36 36
37 37
38 38
31 31
50 50
49 49
68 68
63 63
18 18
48 48
57 57
49 49
42 42
29 29


In [38]:
#for exp in Experimentos:
#    print(exp.flir['espesor'] if exp.flir is not None else None)

In [8]:
#crear GRUPOS segun CAUDAL
from itertools import groupby
count = 0

#key es la lista donde estan los diccionarios de exp y DF
#group es que se agrupa segun el valor de 'caudal' en este caso

for key, group in groupby(sorted(listaporcaudal, key=lambda x: x['caudal']), key=lambda x: x['caudal']):
    print(key)
    count += 1
    #for exp in group:
        #muestra el nombre del grupo y los elementos que tiene
        #print(exp['nombre']) 

1e-06
5e-06
1e-05
1.7e-05
2.5e-05


In [40]:
#ESPESOR vs TIEMPO para 1 caudal a la vez
# 1 cc 6 exp 5 cc 6 exp 10cc 1e-05 6 exp 25cc 2.5e-05 3 exp 5a25xx 1 exp

import matplotlib.lines as mlines
import matplotlib.cm as cm
import matplotlib.mlab as mlab

fig = plt.figure()
axes = []
f = 0

for x in range(6): #range segun el numero de exp por caudal
    axes.append(fig.add_subplot(2, 3, x + 1)) #subplot segun numero de figuras en el plot (3x3, 4x3, 2x3 etc)

for i, (key, group) in enumerate(groupby(sorted(listaporcaudal, key=lambda x: x['caudal']), key=lambda x: x['caudal'])):
    print(i,key)
    c = 0
    if key==1e-06:
        for exp in sorted(group, key=lambda x: x['temperatura']):
            c = c+1
            ax=axes[f]
            min = -50
            max = int(exp['tiempo a'].max()+500)
            ax.set_xlim(min, max)
            #ax.set_ylim(0, maximoejeespesor)
            ax.set_title(exp['nombre'])
            ax.grid(True, which='both', color='gainsboro')

            ax.plot(exp['tiempo a'], exp['espesor a'], linestyle=':', color='royalblue')
            ax.plot(exp['tiempo b'], exp['espesor b'], marker='o', markersize='3', linestyle='None', color='darkorange')
            ax.set_xlabel('Tiempo [s]')
            ax.set_ylabel('Espesor [m]')
            ax.axhline(exp['espesorfinal'], label='Espesor final medido', color='silver')
            ax.axvline(exp['teyec'], label='tiempo fin eyección', color='rosybrown')

            f=f+1
            
plt.suptitle('Evolución de espesores en el tiempo según aproximaciones geométricas para caudales de 1[cc/s]')

elipse = mlines.Line2D([], [], color='royalblue', linestyle=':', label='Aproximación medio elipsoide')
largoxancho = mlines.Line2D([], [], color='darkorange', marker='o',markersize=3, label='Aproximación Largo*Ancho*1/2')
vertical = mlines.Line2D([], [], color='rosybrown', label='Tiempo fin eyección')
horizontal = mlines.Line2D([], [], color='silver', marker='o',markersize=3, label='Espesor final medido')

legend2 = plt.legend(handles=[vertical, horizontal], bbox_to_anchor=(1.8, 0.4), loc=0)
plt.gca().add_artist(legend2)
plt.legend(handles=[elipse, largoxancho], title='Leyenda', loc=1, bbox_to_anchor=(1.8, 1), 
           labels=['Aproximacion medio elipsoide', r'Aproximacion $(X_max*Y_max)/2$'], ncol=1)
fig.show()

0 1e-06
1 5e-06
2 1e-05
3 1.7e-05
4 2.5e-05


In [14]:
#LISTA para AREA vs TIEMPO
otralistaporcaudal = []

for exp in Experimentos:
    if exp.flir is not None:
        try:
            holder = {} #es un diccionario 
            holder['nombre'] = exp.param['NOMBRE'][0]#solo el valor de la primera fila
            holder['area'] = exp.flir['area_m2']#toda la columna
            holder['tiempo'] = exp.flir['FLIR: tiempo']#toda la columna
            holder['caudal'] = exp.param['caudal'][0]#solo el valor de la primera fila
            holder['angulo'] = exp.param['angulo inclinacion'][0]
            holder['uso'] = exp.param['uso manjar'][0]

            otralistaporcaudal.append(holder)
        except:
            continue

In [16]:
#AREA vs TIEMPO agrupados por caudales
fig = plt.figure()
axes = [fig.add_subplot(3,2,i+1) for i in range(count)]
counter = 0

for key, group in groupby(sorted(otralistaporcaudal, key=lambda x: x['caudal']), key=lambda x: x['caudal']):
    print(key)
    ax = axes[counter]
    #ax.set_xlim(minimoejelargo, maximoejelargo)
    #ax.set_ylim(0, maximoejeespesor)
    ax.set_title('{}cc'.format(key*1000000))
    counter += 1
    
    for exp in group:
        ax.plot(exp['tiempo'], exp['area'])
        
#legend1 = plt.legend(handles=[pat1, pat2, pat3, pat4, pat5, pat6, pat7], bbox_to_anchor=(1.8, 0.5), loc=5)
#plt.legend(bbox_to_anchor=(1.8, 0.5), loc=6)
#plt.gca().add_artist(legend1)
fig.show()

1e-06
5e-06
1e-05
1.7e-05
2.5e-05
